In [ ]:
#Author: JasRam
#Prerequesites
#Eikon installed
#!{sys.executable} -m pip install package
#for the following packages (one at a time): dash, dash-core-components, dash-html-components, dash-table, eikon
#Output is a fully functional, interactive webapp running on localhost in under 40 lines of code with no JS

In [ ]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dte
import eikon as ek
from datetime import datetime as dt
ek.set_app_key('YOUR_APP_KEY') #replace with your app key generated from Eikon's APPKEY generator

app = dash.Dash('Hello World')
start = "2016/01/01"
uni, err = ek.get_data(["0#.DJI"], fields = ['TR.CommonName'] )
sl = uni.Instrument.astype(str).values.tolist()
selected_dropdown_value = "IBM.N"

#Initial API Calls
df = ek.get_timeseries(sl, fields=['CLOSE'], start_date=start, interval='daily') #Price History
df2 = ek.get_news_headlines('R:IBM.N') # News
df3, err = ek.get_data(["IBM.N"], fields = ['TR.CompanyMarketCap', 'TR.BetaFiveYear', 'TR.PE', 'TR.DividendYield', 'TR.FreeFloatPct'])

#Layout App Compnents - Inputs, Outputs etc Note pre-population
app.layout = html.Div([
    html.H4('Sample Dash/Eikon Financial App'),
    dcc.Dropdown(
        id='my-dropdown',
        options = [{'label': i, 'value': i} for i in uni.Instrument.unique()],
        value='IBM.N'
    ),
    dcc.Graph(
        id='my-graph'), 
    html.H4('Latest News Headlines'),
    dte.DataTable(id='my-dte',
                  columns=[{"name": i, "id": i} for i in df2.columns],
                  data=df2.to_dict('records')
    ),
    html.H4('Selected Financial Ratios'),
    dte.DataTable(id='my-dte2',
                  columns=[{"name": i, "id": i} for i in df3.columns],
                  data=df3.to_dict('records')
    ),                 
]) 
        
#Manage Callbacks for reactive objects
@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
    return {
        'data': [{
            'x': df.index.astype(str).tolist(),
            'y': df[selected_dropdown_value].values.tolist()
        }],
        'layout': {'margin': {'l': 40, 'r': 0, 't': 20, 'b': 30}}
    }

@app.callback(Output('my-dte', 'data'), [Input('my-dropdown', 'value')])
def update_rows(selected_dropdown_value):
    df2 = ek.get_news_headlines('R:'+selected_dropdown_value)
    return df2.to_dict('records')

@app.callback(Output('my-dte2', 'data'), [Input('my-dropdown', 'value')])
def update_rows2(selected_dropdown_value):
    df3, err = ek.get_data([selected_dropdown_value], fields = ['TR.CompanyMarketCap', 'TR.BetaFiveYear', 'TR.PE', 'TR.DividendYield', 'TR.FreeFloatPct'])
    return df3.to_dict('records')

app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

if __name__ == '__main__':
    app.run_server()